In [ ]:
# Setup chunk to install and load required packages
knitr::opts_chunk$set(warning = FALSE, message = FALSE, include = FALSE)
suppressWarnings(if(!require("pacman")) install.packages("pacman"))

pacman::p_load('tidyverse', 'tidymodels', 'glmnet',
               'randomForest', 'xgboost','patchwork',
               'paletteer', 'here', 'doParallel', 'summarytools')


## Regression

*Supervised* machine learning techniques involve training a model to operate on a set of *features* and predict a *label* by using a dataset that includes some already known label values. The training process fits the features to the known labels to define a general function that can be applied to new features for which the labels are unknown and predict them. You can think of this function as: 

- *y* represents the label you want to predict.
- *x* represents the features the model uses to predict it.

$$y = f(x)$$

In most cases, *x* is a *vector* that consists of multiple feature values. To be more precise, the function could be expressed as:

$$y = f([x_1, x_2, x_3, ...])$$

The goal of training the model is to find a function that performs a calculation to the *x* values that produces the result *y*. You'll apply a machine learning algorithm that tries to fit the *x* values to a calculation that produces *y* reasonably accurately for all the cases in the training dataset.

There are many machine learning algorithms for supervised learning. They can be broadly divided into two types:

- **Regression algorithms**: Algorithms that predict a *y* value that's a numeric value, such as the price of a house or the number of sales transactions.
- **Classification algorithms**: Algorithms that predict to which category, or *class*, an observation belongs. The *y* value in a classification model is a vector of probability values between 0 and 1, one for each class. They indicate the probability of the observation belonging to each class.

In this unit, you'll focus on *regression*. You'll use an example based on a real study. Data for a bicycle-sharing scheme was collected and used to predict the number of rentals based on seasonality and weather conditions. You'll use a simplified version of the dataset from that study.

> **Note:** The data used in this exercise is derived from [Capital Bikeshare](https://www.capitalbikeshare.com/system-data). It's used in accordance with the published [license agreement](https://www.capitalbikeshare.com/data-license-agreement).

### Explore the data

The first step in any machine learning project is to explore the data that you'll use to train a model. The goal is to try to understand the relationships between the attributes. You want to determine any apparent correlation between the features and the label your model will try to predict.

This step might require some work to detect and:

- **Fix issues in the data** like dealing with missing values, errors, or outlier values.
- **Derive new feature columns** by transforming or combining existing features. This process is known as *feature engineering*.
- **Normalize numeric features** so they're on a similar scale. Numeric features are values you can measure or count.
- **Encode categorical features** as numeric indicators. Categorical features are values that represent discrete categories.

Let's load the bicycle-sharing data as a *tibble* and view the first few rows.


In [ ]:
# Load the core tidyverse and make it available in your current R session.
library(tidyverse)

# Import the data into the R session
bike_data <- read_csv(file = "https://raw.githubusercontent.com/MicrosoftDocs/ml-basics/master/data/daily-bike-share.csv", show_col_types = FALSE)

# View first few rows
bike_data %>% 
  slice_head(n = 7)


Sometimes, you might want more information on your data. You can have a look at the data and its structure by using the [*glimpse()*](https://pillar.r-lib.org/reference/glimpse.html) function.



In [ ]:
# Take a quick glance at the data
glimpse(bike_data)


You can observe that `glimpse()` gives you the total number of rows, which contain the observations, and columns, which contain the variables. Then the first few entries of each variable appear in a row after the variable name. The *data type* of the variable is given immediately after each variable's name inside `< >`.

The data consists of 731 rows and the following 14 columns:

-   **instant**: A unique row identifier.
-   **dteday**: The date on which the data was observed. In this case, the data was collected daily, so there's one row per date.
-   **season**: A numerically encoded value that indicates the season. The value 1 represents spring, 2 represents summer, 3 represents fall, and 4 represents winter.
-   **yr**: The year of the study in which the observation was made. The study took place over two years. Year 0 represents 2011, and year 1 represents 2012.
-   **mnth**: The calendar month in which the observation was made. The value 1 represents January and continues through 12 for December.
-   **holiday**: A binary value indicates whether the observation was made on a public holiday.
-   **weekday**: The day of the week on which the observation was made. The value 0 represents Sunday and continues through 6 for Saturday.
-   **workingday**: A binary value indicates whether the day is a working day. It's not a weekend or holiday.
-   **weathersit**: A categorical value indicates the weather situation. The value 1 represents clear, 2 represents mist and clouds, 3 represents light rain and snow, and 4 represents heavy rain, hail, snow, and fog.
-   **temp**: The temperature in Celsius (normalized).
-   **atemp**: The apparent, or "feels-like," temperature in Celsius (normalized).
-   **hum**: The humidity level (normalized).
-   **windspeed**: The windspeed (normalized).
-   **rentals**: The number of bicycle rentals recorded.

In this dataset, *rentals* represents the *label*, which is the $y$ value, your model must be trained to predict. The other columns are potential features. They're $x$ values.

As mentioned previously, you can perform some feature engineering to combine or derive new features. For example, let's add a new column named **day** to the data frame. To add the column, extract the day component from the existing **dteday** column. The new column represents the day of the month from 1 to 31.

From the output of `glimpse()`, you'll realize that the **dteday** column is stored as a *character* vector. So, you'll first need to transform this vector to a date object.

> [Lubridate](https://lubridate.tidyverse.org/), a package in the Tidyverse, provides tools that make it easier to parse and manipulate dates.


In [ ]:
# load lubridate into the R session
library(lubridate)

# Parse dates then extract days
bike_data <- bike_data %>%
  # Parse dates
  mutate(dteday = mdy(dteday)) %>% 
  #Get day
  mutate(day = day(dteday))

# extract the first 10 rows
bike_data %>% 
  slice_head(n = 10)


To start your analysis of the data, you'll examine a few key descriptive statistics. You can use the `summarytools::descr()` function to neatly and quickly summarize the numeric features and the **rentals** label column.


In [ ]:
# load package into the R session
library(summarytools)

# Obtain summary stats for feature and label columns
bike_data %>% 
  # Select features and label
  select(c(temp, atemp, hum, windspeed, rentals)) %>% 
  # Summary stats
  descr(order = "preserve",
        stats = c('mean', 'sd', 'min', 'q1', 'med', 'q3', 'max'),
        round.digits = 6)


The statistics reveal some information about the distribution of the data in each of the numeric fields. The statistics include the number of observations (there are 731 records), the mean, standard deviation, minimum and maximum values, and the quartile values. The quartile values are the threshold values for 25%, 50% (which is also the median), and 75% of the data.

From this information, you can see that the mean number of daily rentals is around 848. There's a comparatively large standard deviation. This deviation indicates a lot of variance in the number of rentals per day.

You might get a clearer idea of the distribution of rental values by visualizing the data. Common plot types for visualizing numeric data distributions are histograms and box plots. Let's use `ggplot2` and create one of each of these plot types for the **rentals** column.

> Patchwork extends the `ggplot` API by providing mathematical operators, such as `+` or `/`, for combining multiple plots.


In [ ]:
library(patchwork)
library(paletteer) # Collection of color palettes
theme_set(theme_light())

# Plot a histogram
hist_plt <- bike_data %>% 
  ggplot(mapping = aes(x = rentals)) + 
  geom_histogram(bins = 100, fill = "midnightblue", alpha = 0.7) +
  
  # Add lines for mean and median
  geom_vline(aes(xintercept = mean(rentals), color = 'Mean'), linetype = "dashed", size = 1.3) +
  geom_vline(aes(xintercept = median(rentals), color = 'Median'), linetype = "dashed", size = 1.3 ) +
  xlab("") +
  ylab("Frequency") +
  scale_color_manual(name = "", values = c(Mean = "red", Median = "yellow")) +
  theme(legend.position = c(0.9, 0.9), legend.background = element_blank())

# Plot a box plot
box_plt <- bike_data %>% 
  ggplot(aes(x = rentals, y = 1)) +
  geom_boxplot(fill = "#E69F00", color = "gray23", alpha = 0.7) +
    # Add titles and labels
  xlab("Rentals")+
  ylab("")


# Combine plots
(hist_plt / box_plt) +
  plot_annotation(title = 'Rental Distribution',
                  theme = theme(plot.title = element_text(hjust = 0.5)))


The plots show that the number of daily rentals ranges from 0 to just over 3,400. The mean (and median) number of daily rentals is closer to the low end of that range. Most of the data is between 0 and around 2,200 rentals. The few values above this range are shown in the box plot as small circles, which indicates they're *outliers*. In other words, they're unusually high or low values beyond the typical range of most of the data.

You can do the same kind of visual exploration of the numeric features. You could use a `for loop`, but ggplot2 lets us use *facets*. With facets, you can create subplots that each display one subset of the data.

Now you need to transform your data into a *long format* by using `tidyr::pivot_longer`, calculate some statistical summaries, and then whip up a histogram for each feature.


In [ ]:
# Create a data frame of numeric features & label
numeric_features <- bike_data %>% 
  select(c(temp, atemp, hum, windspeed, rentals))

# Pivot data to a long format
numeric_features <- numeric_features %>% 
  pivot_longer(!rentals, names_to = "features", values_to = "values") %>%
  group_by(features) %>% 
  mutate(Mean = mean(values),
         Median = median(values))


# Plot a histogram for each feature
numeric_features %>%
  ggplot() +
  geom_histogram(aes(x = values, fill = features), bins = 100, alpha = 0.7, show.legend = F) +
  facet_wrap(~ features, scales = 'free')+
  paletteer::scale_fill_paletteer_d("ggthemes::excel_Parallax") +
  
  # Add lines for mean and median
  geom_vline(aes(xintercept = Mean, color = "Mean"), linetype = "dashed", size = 1.3 ) +
  geom_vline(aes(xintercept = Median, color = "Median"), linetype = "dashed", size = 1.3 ) +
  scale_color_manual(name = "", values = c(Mean = "red", Median = "yellow")) 
  


The numeric features seem to be more *normally* distributed. The mean and median are nearer the middle of the range of values. This position coincides with where the most commonly occurring values are.

> **Note:** The distributions are not *truly normal* in the statistical sense, which would result in a smooth, symmetrical "bell-curve" histogram. The mean and mode, which is the most common value, would be in the center. But they do generally indicate that most of the observations have a value somewhere near the middle.

You've explored the distribution of the *numeric* values in the dataset, but what about the *categorical* features? The categorical features aren't continuous numbers on a scale, so you can't use histograms. You can plot a bar chart that shows the count of each discrete value for each category.

You'll follow the same procedure you used for the numeric features.


In [ ]:
# Create a data frame of categorical features & label
categorical_features <- bike_data %>% 
  select(c(season, mnth, holiday, weekday, workingday, weathersit, day, rentals))

# Pivot data to a long format
categorical_features <- categorical_features %>% 
  pivot_longer(!rentals, names_to = "features", values_to = "values") %>%
  group_by(features) %>% 
  mutate(values = factor(values))


# Plot a bar plot for each feature
categorical_features %>%
  ggplot() +
  geom_bar(aes(x = values, fill = features), alpha = 0.7, show.legend = F) +
  facet_wrap(~ features, scales = 'free') +
  paletteer::scale_fill_paletteer_d("ggthemr::solarized") +
  theme(
    panel.grid = element_blank(),
    axis.text.x = element_text(angle = 90))


Many of the categorical features show a more or less *uniform* distribution. Uniform means there's roughly the same number of rows for each category. Exceptions include:

-   **holiday**: There are many fewer days that are holidays than days that aren't.
-   **workingday**: There are more working days than nonworking days.
-   **weathersit**: Most days are category 1, which represents clear. Category 2, which represents mist and clouds, is the next most common. There are comparatively few category 3 days, which represent light rain or snow. There are no category 4 days at all. This category represents heavy rain, hail, snow, or fog.

You now know something about the distribution of the data in your columns. You can start to look for relationships between the features and the rentals label you want to be able to predict.

For the numeric features, you can create scatter plots that show the intersection of feature and label values.


In [ ]:
# Plot a scatter plot for each feature
numeric_features %>% 
  mutate(corr_coef = cor(values, rentals)) %>%
  mutate(features = paste(features, ' vs rentals, r = ', corr_coef, sep = '')) %>% 
  ggplot(aes(x = values, y = rentals, color = features)) +
  geom_point(alpha = 0.7, show.legend = F) +
  facet_wrap(~ features, scales = 'free')+
  paletteer::scale_color_paletteer_d("ggthemes::excel_Parallax")


The *correlation* statistic, *r*, quantifies the apparent relationship. The correlation statistic is a value between -1 and 1 that indicates the strength of a linear relationship.



In [ ]:
# Calculate correlation coefficient
numeric_features %>% 
  summarise(corr_coef = cor(values, rentals))


The results aren't conclusive. But if you look closely at the scatter plots for `temp` and `atemp`, you can see a vague diagonal trend. This trend shows that higher rental counts tend to coincide with higher temperatures. A correlation value of just over 0.5 for both features supports this observation. Conversely, the plots for `hum` and `windspeed` show a slightly negative correlation. This trend indicates there are fewer rentals on days with high humidity or windspeed.

Now let's compare the categorical features to the label. You'll create box plots that show the distribution of rental counts for each category.


In [ ]:
# Plot a box plot for each feature
categorical_features %>%
  ggplot() +
  geom_boxplot(aes(x = values, y = rentals, fill = features), alpha = 0.9, show.legend = F) +
  facet_wrap(~ features, scales = 'free') +
  paletteer::scale_fill_paletteer_d("tvthemes::simpsons")+
  theme(
    panel.grid = element_blank(),
    axis.text.x = element_text(angle = 90))


The plots show some variance in the relationship between some category values and rentals. For example, there's a clear difference in the distribution of rentals on weekends (*weekday 0 or 6*) and those during the working week (*weekday 1 to 5*). Similarly, there are notable differences for *holiday* and *workingday* categories. A noticeable trend shows different rental distributions in summer and fall months compared to spring and winter months. The *weathersit* category also seems to make a difference in rental distribution. The *day* feature we created for the day of the month shows little variation. It's probably not predictive of the number of rentals.

You've gone through the phase of understanding the data, which is often referred to as exploratory data analysis (EDA). EDA shows how the different variables are related to one another, their distributions, typical ranges, and other attributes. With these insights in mind, it's time to train some regression models.

### Train a regression model by using Tidymodels

Let's use the data to train a regression model that uses the features you've identified as potentially predictive to predict the *rentals* label. You need to create a data frame that contains the predictive features and the label. You also need to specify the roles of the predictors. Are they quantitative predictors like integers or doubles? Or are they nominal predictors like characters or factors?


In [ ]:
# Select desired features and labels
bike_select <- bike_data %>% 
  select(c(season, mnth, holiday, weekday, workingday, weathersit,
           temp, atemp, hum, windspeed, rentals)) %>% 
  # Encode certain features as categorical
  mutate(across(1:6, factor))

# Get a glimpse of your data
glimpse(bike_select)


You could have deselected the unwanted columns by using `select(-c(…))`, but that's an alternative for next time.

You could train a model by using all the data, but it's common practice in supervised learning to split the data into two subsets. Typically, the larger set is used to train the model. A smaller *hold-back* set is used to validate the trained model. Using two subsets enables us to evaluate how well the model performs to get a better estimate of how your models will perform on new data. 

It's important to split the data randomly. For example, you don't want to take the first 70% of the data for training and keep the rest for validation. Random splitting helps to ensure the two subsets of data are statistically comparable. You validate the model with data that has a similar statistical distribution to the data on which it was trained.

To randomly split the data, you'll use `rsample::initial_split()`. The rsample package is one of the many packages in the Tidymodels framework.


In [ ]:
# Load the Tidymodels packages
library(tidymodels)

# Split 70% of the data for training and the rest for testing
set.seed(2056)
bike_split <- bike_select %>% 
  initial_split(prop = 0.7,
  # splitting data evenly on the holiday variable
                strata = holiday)

# Extract the data in each split
bike_train <- training(bike_split)
bike_test <- testing(bike_split)


cat("Training Set", nrow(bike_train), "rows",
    "\nTest Set", nrow(bike_test), "rows")


The result is the following two datasets:

-   **bike_train**: A subset of the dataset used to train the model.
-   **bike_test**: A subset of the dataset used to validate the model.

You're ready to train a model by fitting a suitable regression algorithm to the training data.

Before you embark on more complex machine learning models, it's good practice to build the simplest possible model to get an idea of what's going on. You'll use a *linear regression* algorithm, which is a common starting point for regression. It works by trying to find a linear relationship between the $x$ values and the $y$ label. The resulting model is a function that conceptually defines a line where every possible $x$ and $y$ value combination intersect.

In Tidymodels, you specify models by using `parsnip()`. The goal of [parsnip](https://parsnip.tidymodels.org/) is to provide a tidy, unified interface to models. It can be used to try a range of models by specifying three concepts:

-   Model *type* differentiates models like logistic regression and decision tree models.
-   Model *mode* includes common options like regression and classification. Some model types support either of these options while some only have one mode.
-   Model *engine* is the computational tool that will be used to fit the model. Often these tools are R packages, such as `"lm"` or `"ranger"`.

In Tidymodels, you capture that modeling information in a model specification. Setting up your model specification can be a good place to start.


In [ ]:
# Build a linear model specification
lm_spec <- 
  # Type
  linear_reg() %>% 
  # Engine
  set_engine("lm") %>% 
  # Mode
  set_mode("regression")


After you specify your model, it can be estimated or trained by using the [`fit()`](https://tidymodels.github.io/parsnip/reference/fit.html) function. Typically, you use a symbolic description of the model, known as a formula, and some data.

And `rentals ~ .` means you'll fit `rentals` as the predicted quantity, explained by all the predictors/features (`.`).


In [ ]:
# Train a linear regression model
lm_mod <- lm_spec %>% 
  fit(rentals ~ ., data = bike_train)

# Print the model object
lm_mod


These are the coefficients that the model learned during training.

### Evaluate the trained model

It's time to see how the model performed.

How do you do this? Now that you've trained the model, you can use it to predict rental counts for the features you held back in your validation dataset by using `parsnip::predict()`. For example:


In [ ]:
# Make predictions on test set
pred <- lm_mod %>% 
  predict(new_data = bike_test)

# View predictions
pred %>% 
  slice_head(n = 5)


To make it easier to perform subsequent operations, you'll combine the test data and predictions in a tibble.
Then you can compare these predictions to the actual label values to evaluate how well, or not, the model is working.


In [ ]:
# Predict rentals for the test set and bind it to the test_set
results <- bike_test %>% 
  bind_cols(lm_mod %>% 
    # Predict rentals
    predict(new_data = bike_test) %>% 
      rename(predictions = .pred))

# Compare predictions
results %>% 
  select(c(rentals, predictions)) %>% 
  slice_head(n = 10)


Comparing each prediction with its corresponding "ground truth" actual value isn't an efficient way to determine how well the model is predicting. Let's see if you can get a better indication by visualizing a scatter plot that compares the predictions to the actual labels. You'll also overlay a trend line to get a general sense of how well the predicted labels align with the true labels.



In [ ]:
# Visualise the results
results %>% 
  ggplot(mapping = aes(x = rentals, y = predictions)) +
  geom_point(size = 1.6, color = "steelblue") +
  # Overlay a regression line
  geom_smooth(method = "lm", se = F, color = 'magenta') +
  ggtitle("Daily Bike Share Predictions") +
  xlab("Actual Labels") +
  ylab("Predicted Labels") +
  theme(plot.title = element_text(hjust = 0.5))


There's a definite diagonal trend. The intersections of the predicted and actual values are generally following the path of the trend line. But there's a fair amount of difference between the ideal function represented by the line and the results. This variance represents the *residuals* of the model. In other words, the difference between the label predicted when the model applies the coefficients it learned during training to the validation data and the actual value of the validation label. These residuals when evaluated from the validation data indicate the expected level of error when the model is used with new data for which the label is unknown.

You can quantify the residuals by calculating commonly used evaluation metrics. You'll focus on the following three:

- **Mean square error (MSE)**: The mean of the squared differences between predicted and actual values. The MSE yields a relative metric in which the smaller the value, the better the fit of the model.
- **Root mean square error (RMSE)**: The square root of the MSE. The RMSE yields an absolute metric in the same unit as the label. In this case, it's the number of rentals. The smaller the value, the better the model. In a simplistic sense, it represents the average number of rentals by which the predictions are wrong.
- **Coefficient of determination (usually known as R-squared or R2)**: A relative metric in which the higher the value, the better the fit of the model. This metric represents how much of the variance between predicted and actual label values the model can explain.

> The `yardstick` package in Tidymodels is used to estimate how well models are working based on the predictions it made for the validation data. For more information about these metrics and other metrics for evaluating regression models, see the [Metric types documentation](https://yardstick.tidymodels.org/articles/metric-types.html).


In [ ]:
# Multiple regression metrics
eval_metrics <- metric_set(rmse, rsq)

# Evaluate RMSE, R2 based on the results
eval_metrics(data = results,
             truth = rentals,
             estimate = predictions)


Now you've quantified the ability of your model to predict the number of rentals. It has some predictive power, but you can probably do better.

### Summary

Here you've explored your data and fit a basic regression model. In the next unit, you'll try other regression algorithms to improve performance.

### Further reading

To learn more about Tidymodels, see the [Tidymodels documentation](https://www.tidymodels.org/).
